# RAG Application using Ollama and LangChain

This notebook demonstrates building a Retrieval-Augmented Generation (RAG) application using:
- LangChain for orchestration
- Ollama for local LLM inference
- ChromaDB for vector storage
- Text embeddings for semantic search

## What is RAG?
RAG combines the capabilities of large language models with external knowledge retrieval, allowing AI systems to access up-to-date information beyond their training data.

## Installation

In [ ]:
!pip3 install langchain
!pip3 install langchain-core
!pip3 install langchain-ollama
!pip3 install langchain_community
!pip3 install langchain-chroma

## LangChain Basics with Ollama

### Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that gives a one-line definition of the word entered by user"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(user_input="Sesquipedalian")
messages

### ChatOllama Integration

In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0
)

In [ ]:
ai_msg = llm.invoke(messages)
ai_msg

### Creating Chains with Output Parsers

In [ ]:
from langchain_core.output_parsers import StrOutputParser
chain = chat_template | llm | StrOutputParser()

In [ ]:
chain.invoke({"user_input": "granny"})

## Building a RAG Application

Now let's build a complete RAG pipeline with the following components:
1. Document Loader
2. Text Splitter
3. Embeddings
4. Vector Store (ChromaDB)
5. Retriever
6. RAG Chain

### Import Required Components

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_ollama import ChatOllama

### Step 1: Load Documents

In [ ]:
raw_documents = TextLoader("./LangchainRetrieval.txt").load()

In [ ]:
raw_documents

### Step 2: Split Documents into Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
len(documents)

In [ ]:
print(documents[0])
print(documents[1])

### Step 3: Create Embeddings Model

In [ ]:
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

### Step 4: Create Vector Store (ChromaDB)

In [ ]:
db = Chroma.from_documents(documents, embedding=oembed)

### Test Similarity Search

In [ ]:
query = "What is text embedding and how does langchain help in doing it"
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
print(docs[3].page_content)

### Step 5: Build RAG Chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
model = ChatOllama(
    model="llama3.1:8b",
    temperature=0
)

In [ ]:
retriever = db.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [ ]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

### Step 6: Use the RAG Chain

In [ ]:
chain.invoke("What is text embedding and how does langchain help in doing it")

## Summary

In this notebook, we've built a complete RAG application that:
1. Loads documents from text files
2. Splits them into manageable chunks
3. Creates embeddings using Ollama's nomic-embed-text model
4. Stores embeddings in ChromaDB vector database
5. Retrieves relevant context based on user queries
6. Generates accurate answers using the Llama 3.1 model

This approach allows the LLM to answer questions based on your custom documents rather than relying solely on its training data.